# Multilayer Perceptron

In our [previous attempt](./02-logistic-regression-word2vec.ipynb), we tried using Word2Vec to improve our sentiment classification, but instead of a higher score, we got a much, much worse result. 

That happened because our existing architecture (logistic regression) was unfit for a new vectorization (seemingly much better) approach. But what if we change the architecture itself?

That gives us a nice opportunity to try a different paradigm called **deep learning** - a branch of machine learning and artificial intelligence that uses artificial neural networks to process data and learn patterns. These networks, inspired by the structure of the human brain, are built with multiple layers of interconnected nodes (**neurons**) that allow them to identify complex relationships and make predictions. 

##  Data Preparation

In [1]:
import kagglehub
df = kagglehub.dataset_load(
    kagglehub.KaggleDatasetAdapter.PANDAS,
    'jp797498e/twitter-entity-sentiment-analysis',
    'twitter_training.csv',
    pandas_kwargs={'encoding': 'ISO-8859-1'},
)

df = df[df.columns[[2, 3]]]
df.columns = ['sentiment', 'text']

df['text'] = df['text'].astype(str)
df['sentiment'] = df['sentiment'].astype(str)
df = df.dropna()

df = df.loc[df['sentiment'] != 'Irrelevant']
display(df)

,sentiment,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...
...,...,...
74676,Positive,Just realized that the Windows partition of my...
74677,Positive,Just realized that my Mac window partition is ...
74678,Positive,Just realized the windows partition of my Mac ...
74679,Positive,Just realized between the windows partition of...


In [2]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
x_train = train['text']
y_train = train['sentiment']
x_test = test['text']
y_test = test['sentiment']

## Semantic Vectorization

Our vectorization routine also remains exactly the same. We are doing this so we can see how the change of *approach* affects the final result with the same data.

In [3]:
import kagglehub
path = kagglehub.dataset_download(
    'leadbest/googlenewsvectorsnegative300', 
    path='GoogleNews-vectors-negative300.bin.gz'
)

from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format(path, binary=True)

import numpy as np
from gensim.utils import simple_preprocess
def vectorize(text):
    tokens = simple_preprocess(text.lower(), deacc=True)
    token_vectors = [wv.get_vector(x) for x in tokens if x in wv]
    if token_vectors:
        return np.mean(token_vectors, axis=0)
    else:
        return np.zeros(wv.vector_size)

x_train = np.array(train['text'].map(vectorize).tolist())
x_test = np.array(test['text'].map(vectorize).tolist())

## Label Encoding

Before we proceed further, we need to transform our output training as well! That happens because neural networks do not work with text *directly* - instead, we need to encode our labels, turning them into some kind of mathematical representation.

In [4]:
from sklearn.preprocessing import LabelBinarizer 
import pandas as pd

encoder = LabelBinarizer()
encoder.fit(df['sentiment'])

y_train_encoded = pd.DataFrame(encoder.fit_transform(y_train))
y_test_encoded = pd.DataFrame(encoder.transform(y_test))

This method is called "one-hot encoding", where each category is assigned a unique binary column. We have three categories - so our encodings will have a dimension of three each.

In [5]:
display(y_train_encoded)

,0,1,2
0,1,0,0
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1
...,...,...,...
49347,0,1,0
49348,0,1,0
49349,1,0,0
49350,0,0,1


## Building and Training the Model

Now, let's design our model structure. This time, we will use a thing called **multilayer perceptron**. As its name states, it is a neural network that consists of multiple **layers** of neurons - allowing one to learn complex, non-linear relationships in data (unlike linear regression classifier).

For this task, we will use three types of layers - input (transforms our source data and passes it next), dense (simple layer of interconnected neurons), and dropout (special layer that helps against overfitting by randomly disabling part of the previous layer during the training process).

In [6]:
from tensorflow.keras import layers, Sequential
num_classes = len(encoder.classes_)
model = Sequential([
    layers.Input(shape=(wv.vector_size,)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(num_classes, activation='softmax')
])

Originally, there were only three layers - input, hidden, and output. But stacking two more hidden layers helped to uplift the original 82% accuracy to 85%. That happened due to increased model capacity, and (potential) hierarchical feature learning.

We can compile and train our model now. Sticking to the CPU may be a good idea for now - using GPU for such a small model may actually slow down the training process due to a huge data transfer overhead.

In [7]:
from tensorflow import device
with device('/CPU:0'):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train_encoded, epochs=50, batch_size=32, validation_split=0.1) 

Epoch 1/50
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5869 - loss: 0.9017 - val_accuracy: 0.6696 - val_loss: 0.7869
Epoch 2/50
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6685 - loss: 0.7735 - val_accuracy: 0.6872 - val_loss: 0.7375
Epoch 3/50
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6939 - loss: 0.7171 - val_accuracy: 0.7052 - val_loss: 0.7021
Epoch 4/50
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7148 - loss: 0.6740 - val_accuracy: 0.7016 - val_loss: 0.7010
Epoch 5/50
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7309 - loss: 0.6325 - val_accuracy: 0.7322 - val_loss: 0.6555
Epoch 6/50
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7548 - loss: 0.5895 - val_accuracy: 0.7411 - val_loss: 0.6228
Epoch 7/50
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7678 - loss: 0.5611 - val_accuracy: 0.7486 - val_loss: 0.5982
Epoch 8/50
1388/1388 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7821 - loss: 0.5269 - 

## Result

In [9]:
from sklearn.metrics import classification_report
with device('/CPU:0'):
    y_pred_probs = model.predict(x_test, verbose=False)
    y_pred_labels = np.argmax(y_pred_probs, axis=1)
    y_true_labels = np.argmax(y_test_encoded.to_numpy(), axis=1)
    print(classification_report(y_true_labels, y_pred_labels, target_names=encoder.classes_))

              precision    recall  f1-score   support

    Negative       0.86      0.86      0.86      4471
     Neutral       0.85      0.80      0.82      3666
    Positive       0.83      0.87      0.85      4202

    accuracy                           0.85     12339
   macro avg       0.85      0.84      0.84     12339
weighted avg       0.85      0.85      0.85     12339



## Conclusion

This experiment yielded an **85%** accuracy, confirming that a deeper, non-linear model can extract more signal from these embeddings than a linear classifier. The addition of hidden layers and dropout underscores the "more capacity, better results" principle, at least to a point. 

While a significant improvement over the 65% Logistic Regression baseline with these features, it still trails the 91% n-gram model. This suggests that to fully leverage Word2Vec semantic richness, architectures capable of processing sequences are the necessary next step.